In [2]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.tree import export_graphviz
df = pd.read_csv('train.csv')

In [3]:
#各変数のNaNの個数のチェック
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
class_mapping_1 = {'male': 0, 'female': 1}
class_mapping_2 = {'C': 0, 'Q': 1,'S':2}
df['Sex'] = df['Sex'].map(class_mapping_1)
df['Embarked'] = df['Embarked'].map(class_mapping_2)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,2.0


In [6]:
#Age,Embarkedに含まれるNaNを平均値で補完
df=df.fillna(df.Age.mean())
df=df.fillna(df.Embarked.mean())
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,29.6991,2.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,29.6991,2.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,29.6991,2.0


In [7]:
#特にSurvivedに関係のなさそうな変数の削除
df2=df.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)

In [8]:
df2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,2.0
1,1,1,1,38.0,1,0,71.2833,0.0
2,1,3,1,26.0,0,0,7.9250,2.0
3,1,1,1,35.0,1,0,53.1000,2.0
4,0,3,0,35.0,0,0,8.0500,2.0


In [9]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
X_1 = df2.iloc[:,1:8]
y_1 = df2['Survived']
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.3, random_state=0)

In [17]:
#勾配ブーストでやるためにグリッドサーチにかける
param_grid = {'max_depth': [1, 2, 3, 4, 5, 6],'learning_rate': [ 0.01, 0.1,0.5,0.8, 1,5, 10],'n_estimators':[10,100,300]}
# 層化
from sklearn.model_selection import StratifiedKFold
kf_5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [19]:
gs_GBC= GridSearchCV(GradientBoostingClassifier(), param_grid, cv=kf_5)

In [20]:
gs_GBC.fit(X_1_train, y_1_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
       error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6], 'learning_rate': [0.01, 0.1, 0.5, 0.8, 1, 5, 10], 'n_estimators': [10, 100, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [22]:
gs_GBC.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}

In [23]:
gs_GBC.score(X_1_test, y_1_test)

0.84328358208955223

In [30]:
gbct = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1,n_estimators=100)

In [31]:
gbct.fit(X_1_train, y_1_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [36]:
dftest=pd.read_csv('test.csv')
class_mapping_1 = {'male': 0, 'female': 1}
class_mapping_2 = {'C': 0, 'Q': 1,'S':2}
dftest['Sex'] = dftest['Sex'].map(class_mapping_1)
dftest['Embarked'] = dftest['Embarked'].map(class_mapping_2)

In [37]:
dftest=dftest.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
dftest=dftest.fillna(dftest.Age.mean())
dftest=dftest.fillna(dftest.Embarked.mean())

In [38]:
gbct.predict_proba(dftest)

array([[ 0.88049538,  0.11950462],
       [ 0.91366713,  0.08633287],
       [ 0.98129201,  0.01870799],
       [ 0.84023114,  0.15976886],
       [ 0.53442554,  0.46557446],
       [ 0.60173084,  0.39826916],
       [ 0.57062126,  0.42937874],
       [ 0.81943814,  0.18056186],
       [ 0.08442931,  0.91557069],
       [ 0.93806293,  0.06193707],
       [ 0.89697492,  0.10302508],
       [ 0.86691026,  0.13308974],
       [ 0.03806656,  0.96193344],
       [ 0.97686018,  0.02313982],
       [ 0.12914327,  0.87085673],
       [ 0.07387773,  0.92612227],
       [ 0.86887051,  0.13112949],
       [ 0.83015706,  0.16984294],
       [ 0.50160217,  0.49839783],
       [ 0.76820086,  0.23179914],
       [ 0.87436036,  0.12563964],
       [ 0.61407471,  0.38592529],
       [ 0.06991106,  0.93008894],
       [ 0.69657047,  0.30342953],
       [ 0.01677501,  0.98322499],
       [ 0.91571563,  0.08428437],
       [ 0.05163958,  0.94836042],
       [ 0.83015706,  0.16984294],
       [ 0.62172518,

In [39]:
gbct.predict_proba(dftest).shape

(418, 2)

In [40]:
def answer(model,n):
    arr= np.array( [] )
    for i in range(0,n):
        if model[i,0]>=0.5:
            arr =np.append( arr, 0)
        else:
            arr =np.append( arr, 1 )
    return arr

In [41]:
a=answer(gbct.predict_proba(dftest),418)

In [42]:
A = pd.DataFrame(a)

In [43]:
A.to_csv('answer2.csv')